# ETL作業第二階段

爬104職位網頁

In [8]:
    from bs4 import BeautifulSoup
    import requests
    import bs4
    import time
    import re
    import sqlite3
    import pprint

In [12]:
# 定義 job104_for_info 方法
# 參數為要爬的網頁的網址
# 爬網頁之後解析出內文並回存進資料庫
# 假設所需的套件都已經啟用

def job104_for_info(href): 

#     from bs4 import BeautifulSoup
#     import requests
#     import bs4
#     import time
#     import re
#     import sqlite3

    # 爬進資料
    # 用lxml出現錯誤,得用Python預設的html.parser

    try:

        time.sleep(5)
        
        res = requests.get(href)
        soup = BeautifulSoup(res.text,'html.parser')
        
        if soup.select('head > title') != "104人力銀行─錯誤頁":

            # 爬進資料
            # 用lxml會出現錯誤,得用Python預設的html.parser

            res = requests.get(href)
            soup = BeautifulSoup(res.text,'html.parser')

            # 拆解資料

            # 工作內容 (job_desc)

            job_desc = soup.select('div[class="content"] > p')[0].text

            # 擅長工具 (job_tools)
            # 工作技能 (job_skills)
            # 其他條件 (other_cons)

            reqs = soup.find_all(["dt","dd"])

            job_tools = ""
            job_skills = ""
            other_cons = ""

            for i in range(0,len(reqs)-1):
                if "擅長工具" in reqs[i].text:
                    job_tools += reqs[i+1].text
                elif "工作技能" in reqs[i].text:
                    job_skills += reqs[i+1].text
                elif "其他條件" in reqs[i].text:
                    other_cons += reqs[i+1].text

            # 將找到的資料都串起來        

            new_info = job_desc + "," + job_tools + "," + job_skills + "," + other_cons

            for char in ['\n','\r','、','，']:
                if char in new_info:
                    new_info = new_info.replace(char,' ')

            new_info = new_info.encode('ascii','ignore').decode('utf8')
            # print(new_info)

            # 將資料上傳到資料庫裡

            with sqlite3.connect('job.sqlite') as conn:
                c = conn.cursor()
                save_info = "update job104 set info = ? where href = ?;" 
                c.execute(save_info,(new_info,href))
    
    except IndexError as e:
        
        #if requests.status_code !=200: 
        #如果網頁異常則拋出例外
        
        print(e, href)

    except:
        print(href)
        
#爬蟲例外 例外方法寫在def裏面
        
class CrawlerError(Exception): 
    pass

In [10]:
# 從資料庫找出需要抓的網頁的網址
# 然後一筆一筆抓
# 存進資料庫

def update_None_info(): 
       
    from bs4 import BeautifulSoup
    import requests
    import bs4
    import time
    import re
    import sqlite3
    
    try:
        with sqlite3.connect('job.sqlite') as conn:

            c = conn.cursor()
            
            # 找出info是空的url並找出網址list
            
#             qryString = "SELECT href FROM job104 where info is Null" 
            qryString = "SELECT href FROM job104 where jobID <= 3000" 
            
            c.execute(qryString)
            
            # 計算有多少筆完成
            
            do_number = 0 
            
            lst = c.fetchall()
            
            # 控制爬網最大比數
            
            lst[0:1000]    
            
            #跑迴圈執行
            
            for a in lst:
                # https://hunter.104.com.tw/TC/services/service_b_introduction.jsp
                if "hunter.104.com.tw" not in a[0]: #開頭網址不是獵頭網的才進入迴圈
                    job104_for_info(a[0]) 
#                     print(a[0])
                    do_number +=1
#                     print(do_number)
#                     if do_number % 10 == 0:
#                         print(do_number)
                else:
                    print("獵頭網頁") #若出現則顯示外包網頁
                
                # 每100筆就休息一次
                
                if do_number % 100 == 0:
                    time.sleep(20)
                
            print('Has crawled {} info!'.format(do_number))
            
            conn.close()
            
    except ConnectionError as e:
        print(e)
        print(a)
        print(href)
        
    except CrawlerError as e:
        print(a)
        print(e)
    finally:
        conn.close()

In [ ]:
#執行方法

update_None_info()

In [11]:
#存進資料庫

with sqlite3.connect('job.sqlite') as conn: 
    c = conn.cursor()
    pprint.pprint(list(c.execute('select * from job104 where jobid >= 2900 and jobid <= 3000;')))
# conn.close()

[(2900,
  '【家教】程式語言-社會人士',
  'https://www.104.com.twhttp://tutor.104.com.tw/trans_job_to_case.cfm?frompage=joblist&jobno=9087756',
  11,
  None),
 (2901,
  '韌體設計高級工程師',
  'https://www.104.com.tw/job/?jobno=5espd&jobsource=n104bank1&hotjob_chr=',
  2,
  None),
 (2902,
  '【G_S】資深數值工程師',
  'https://www.104.com.tw/job/?jobno=5esyt&jobsource=n104bank1&hotjob_chr=',
  2,
  '1. Excel2. (C/C++C#)3.  RTP4.   PLUS1.2.SLOT3. ,C C++ Matlab R Excel,,'),
 (2903,
  '網路工程人員',
  'https://www.104.com.tw/job/?jobno=5eusn&jobsource=n104bank1&hotjob_chr=',
  3,
  '1.ASP.NET MSSQL IIS 2.  3. 4. 5. 6.,ASP C# DHTML HTML JavaScript XML jQuery '
  'IIS,,1.ASP.NET IIS MSSQL 2. 3.   '),
 (2904,
  'ERP/WMS 資深工程師',
  'https://www.104.com.tw/job/?jobno=5euti&jobsource=n104bank1&hotjob_chr=',
  7,
  '1.ERP/WMS  2. 3. 4..net(C# OR VB)MS SQL 5.,,,'),
 (2905,
  'Sales and Client Service Technology Application Developer (約聘)',
  'https://www.104.com.tw/job/?jobno=5ev4b&jobsource=n104bank1&hotjob_chr=',
  7,
  'Descriptio

# 加入單筆

In [ ]:
jid = 1

In [ ]:
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
#     c.execute('select * from job518')
#     print(c.fetchone())
    updStrting = 'update job518 set info =:info where jobID =:jobID'
    c.execute(updStrting, {'info' : new_info, 'jobID' : jid})
    

In [ ]:
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
    c.execute('select * from job518')
    print(c.fetchone())